Question 1: Extract Tesla stock data using yfinance

In [63]:
!pip install yfinance==0.2.4
!pip install bs4

In [84]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [62]:
tesla = yf.Ticker("TSLA")
tesla_stock_info=tesla.history(period="1mo")

In [53]:
tesla_stock_info.reset_index(inplace=True)
tesla_stock_info.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-09-06 00:00:00-04:00,255.139999,255.389999,245.059998,251.919998,116959800,0.0,0.0
1,2023-09-07 00:00:00-04:00,245.070007,252.809998,243.270004,251.490005,115312900,0.0,0.0
2,2023-09-08 00:00:00-04:00,251.220001,256.519989,246.669998,248.500000,118367700,0.0,0.0
3,2023-09-11 00:00:00-04:00,264.269989,274.850006,260.609985,273.579987,174667900,0.0,0.0
4,2023-09-12 00:00:00-04:00,270.760010,278.390015,266.600006,267.480011,135999900,0.0,0.0


REASONING FOR WHY MY PERIOD IS SET TO 1mo: THE PERIOD='MAX' argument returns an empty dataframe as there is no data apparently, hence i have opted to set period='1mo' in my answer which i hope is fine as there was no specification in the question.

In [ ]:
#tesla = yf.Ticker("TSLA")
#tesla_stock_info=tesla.history(period="max")
#tesla_stock_info.reset_index(inplace=True)
#tesla_stock_info.head()

Question 2: Use webscraping to extract Tesla revenue data

In [101]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
html_data=requests.get(url).text
soup=BeautifulSoup(html_data, 'html5lib')
table=soup.find_all("tbody")[1]     #find the table
tesla_revenue = pd.DataFrame(columns=['Date','Revenue'])             #create empty datframe

for row in table.find_all('tr'):
    col= row.find_all("td")     #find all the td tags
    Date=col[0].text     #convert html tag to non-programmer readable script
    Revenue=col[1].text   
    tesla_revenue.loc[len(tesla_revenue.index)] = [Date, Revenue]        #add a row of revenue every loop    
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(',|\$',"")
tesla_revenue.dropna(inplace=True)
tesla_revenue=tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.tail()

,Date,Revenue
48,2010-09-30,$31
49,2010-06-30,$28
50,2010-03-31,$21
52,2009-09-30,$46
53,2009-06-30,$27


Question 3: Extract stock data for GME

In [12]:
gme= yf.Ticker("GME")
gme_stock = gme.history(period='1mo')
gme_stock.reset_index(inplace=True)
gme_stock.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-09-06 00:00:00-04:00,19.059999,19.15,18.420000,18.750000,6107200,0.0,0.0
1,2023-09-07 00:00:00-04:00,19.129999,19.48,17.610001,18.889999,5021300,0.0,0.0
2,2023-09-08 00:00:00-04:00,18.150000,18.50,17.350000,17.709999,4689200,0.0,0.0
3,2023-09-11 00:00:00-04:00,17.850000,17.85,16.900000,17.020000,3543800,0.0,0.0
4,2023-09-12 00:00:00-04:00,17.530001,18.50,17.190001,17.230000,5179400,0.0,0.0


Question 4: Use webscraping to extract GME Revenue data

In [100]:
url1='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
data=requests.get(url1).text
soup1=BeautifulSoup(data)
table1=soup.find_all("tbody")[1]
GME_Revenue=pd.DataFrame(columns=['Date','Revenue'])

for row in table1.find_all('tr'):
    col= row.find_all("td")   
    Date=col[0].text     
    Revenue=col[1].text   
    GME_Revenue.loc[len(GME_Revenue.index)] = [Date, Revenue]            
GME_Revenue["Revenue"] = GME_Revenue["Revenue"].str.replace(',|\$',"")
GME_Revenue.dropna(inplace=True)
GME_Revenue=GME_Revenue[GME_Revenue['Revenue'] != ""]
GME_Revenue.tail()                                   

,Date,Revenue
48,2010-09-30,$31
49,2010-06-30,$28
50,2010-03-31,$21
52,2009-09-30,$46
53,2009-06-30,$27


Question 5: Plot stock data for Tesla

In [105]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [103]:
make_graph(tesla_stock_info,tesla_revenue, 'Tesla')

AttributeError: 'DataFrame' object has no attribute 'Date'

Question 6: Plot Stock data for GME

In [104]:
make_graph(gme_stock,GME_Revenue, 'GME')

C:\Users\rahil\AppData\Local\Temp\ipykernel_18820\2068038883.py:5: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

C:\Users\rahil\AppData\Local\Temp\ipykernel_18820\2068038883.py:6: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.



ValueError: could not convert string to float: '$10,389'